In [1]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 498.1 kB/s eta 0:00:0000:0100:01


# MNI Calculation

This notebook prepares data for MNI calculation and writes the results to the `source_data.xlsx` file.

In [10]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from scripts.mni import calculate_mni

# Load source data and normalise column names
df_occurrences = pd.read_pickle('../data/pkl/df_occurrences.pkl')
df_briana_with_responses = pd.read_pickle('../data/pkl/df_briana_with_responses.pkl')
df_occurrences.columns = df_occurrences.columns.str.strip()
df_briana_with_responses.columns = df_briana_with_responses.columns.str.strip()

# Coerce identifiers to integers
df_occurrences['ID'] = pd.to_numeric(df_occurrences['ID'], errors='coerce').astype('Int64')
df_briana_with_responses['OccurrenceID'] = pd.to_numeric(df_briana_with_responses['OccurrenceID'], errors='coerce').astype('Int64')

# Ensure a populated 'Taxon Label' column exists
if 'Taxon Label' not in df_occurrences.columns or df_occurrences['Taxon Label'].isna().all():
    df_occurrences['Taxon Label'] = pd.NA
    for c in ['Post: Taxon Guess?', 'Pre: Taxon']:
        if c in df_occurrences.columns:
            df_occurrences['Taxon Label'] = df_occurrences['Taxon Label'].fillna(df_occurrences[c])

# Select required columns from each table
columns_occurrences = ['ID', 'TransectUID', 'Taxon Label', 'Pre: Sex', 'Pre: Age']
columns_briana = ['OccurrenceID', 'Weathering class', 'What element is this?', 'Side', 'Complete', 'Complete?']
df_occurrences = df_occurrences.reindex(columns=columns_occurrences)
df_briana_with_responses = df_briana_with_responses.reindex(columns=columns_briana)

# Fill missing 'Complete?' values with fallback from 'Complete'
df_briana_with_responses['Complete?'] = (
    df_briana_with_responses['Complete?']
    .replace(r'^\s*$', pd.NA, regex=True)
    .fillna(df_briana_with_responses['Complete'])
)
df_briana_with_responses = df_briana_with_responses.drop(columns=['Complete'])

# Merge and keep OccurrenceID for later grouping
df = (
    df_occurrences
    .merge(
        df_briana_with_responses,
        left_on='ID',
        right_on='OccurrenceID',
        how='left',
    )
    .drop(columns=['ID'])
)
df['TransectUID'] = pd.to_numeric(df['TransectUID'], errors='coerce').astype('Int64')

# For each OccurrenceID, retain the highest weathering class
import re
def _wx_score(val):
    nums = re.findall(r'\d+', str(val))
    return max(map(int, nums)) if nums else -1
df['Weathering class'] = (
    df.groupby('OccurrenceID')['Weathering class']
    .transform(lambda s: s.loc[s.map(_wx_score).idxmax()])
)

# Keep all completed rows; for incomplete elements, drop duplicate entries across key fields
non_no = df[df['Complete?'] != 'No']
subset_cols = [
    'TransectUID',
    'OccurrenceID',
    'Taxon Label',
    'Pre: Sex',
    'Pre: Age',
    'Weathering class',
    'What element is this?',
    'Side',
]
no_rows = df[df['Complete?'] == 'No'].drop_duplicates(subset=subset_cols)
df = pd.concat([non_no, no_rows], ignore_index=True)

# Drop helper columns
df = df.drop(columns=['OccurrenceID', 'Complete?'])

# Remove high-level and other taxa not needed for MNI
df = df[~df['Taxon Label'].str.lower().isin(['mammalia indet', 'ungulate', 'ostrich', 'Aves (medium)', 'Aves (small)'])]



In [11]:
# Pivot side counts by transect, taxon, sex, age, weathering class, and element
pivot_df = (
    df.pivot_table(
        index=["TransectUID", "Taxon Label", "Pre: Sex", "Pre: Age", "Weathering class", "What element is this?"],
        columns="Side",
        aggfunc="size",
        fill_value=0,
    )
    .rename_axis(columns=None)
    .reset_index()
)
pivot_df["TransectUID"] = pd.to_numeric(pivot_df["TransectUID"], errors="coerce").astype("Int64")

# Split counts for observations with unknown side evenly between left and right
if "unknown" in pivot_df.columns:
    pivot_df["unknown"] = np.ceil(pivot_df["unknown"] / 2).astype(int)

pivot_df.head()
from pathlib import Path
pivot_output_path = Path("../data/export/excel/pivot_df.xlsx")
pivot_output_path.parent.mkdir(parents=True, exist_ok=True)
pivot_df.to_excel(pivot_output_path, index=False)


In [12]:
mni_per_transect, group_mni = calculate_mni(pivot_df)
mni_per_transect
mni_per_transect_output_path = Path("../data/export/excel/mni_per_transect_df.xlsx")
mni_per_transect_output_path.parent.mkdir(parents=True, exist_ok=True)
mni_per_transect.to_excel(mni_per_transect_output_path, index=False)


In [13]:
from pathlib import Path

output_path = Path("../data/export/excel/source_data.xlsx")
if output_path.exists():
    transect_sheet = pd.read_excel(output_path, sheet_name="Transects")
else:
    transect_sheet = pd.DataFrame(columns=["TransectUID"])

if "TransectUID" not in transect_sheet.columns:
    if "UID" in transect_sheet.columns:
        transect_sheet = transect_sheet.rename(columns={"UID": "TransectUID"})
    else:
        transect_sheet["TransectUID"] = pd.NA

transect_sheet["TransectUID"] = pd.to_numeric(transect_sheet["TransectUID"], errors="coerce").astype("Int64")
mni_per_transect["TransectUID"] = pd.to_numeric(mni_per_transect["TransectUID"], errors="coerce").astype("Int64")

transect_sheet = transect_sheet.merge(mni_per_transect, on="TransectUID", how="left")
transect_sheet = transect_sheet.rename(columns={"MNI": "MNI_calc"})

output_path.parent.mkdir(parents=True, exist_ok=True)
if output_path.exists():
    with pd.ExcelWriter(output_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
        transect_sheet.to_excel(writer, sheet_name="Transects", index=False)
        group_mni.to_excel(writer, sheet_name="Group MNI", index=False)
else:
    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        transect_sheet.to_excel(writer, sheet_name="Transects", index=False)
        group_mni.to_excel(writer, sheet_name="Group MNI", index=False)


In [14]:
from pathlib import Path
import numbers

EXCLUDED_MNI_TAXA = {"Aves (medium)", "Aves (small)", "Mammalia indet.", "Ungulata"}
habitats = ["grass closed", "shrubs open", "shrubs closed", "trees closed"]
total_label = "eastern OPC"

# Load pickled data
df_transects = pd.read_pickle('../data/pkl/df_transects.pkl')

# Filter transects and species (Old Reserve only)
df_transects['Year'] = pd.to_datetime(df_transects['start_time']).dt.year
mask_not_2008 = df_transects['Year'] != 2008
mask_keep_2024 = ~((df_transects['Year'] == 2024) & (df_transects['Pre: Transect physical habitat'] != 'shrubs closed'))
df_filtered_transects = df_transects[mask_not_2008 & mask_keep_2024].copy()

df_filtered_transects.columns = df_filtered_transects.columns.str.strip()
transect_meta = (
    df_filtered_transects[["UID", "Pre: On old reserve?", "Pre: Transect physical habitat"]]
    .rename(columns={"UID": "TransectUID"})
    .dropna(subset=["TransectUID"])
)
transect_meta["TransectUID"] = pd.to_numeric(
    transect_meta["TransectUID"], errors="coerce"
).astype("Int64")
transect_meta["habitat"] = (
    transect_meta["Pre: Transect physical habitat"].astype(str).str.strip().str.lower()
)

def _safe_int(value):
    return int(value) if pd.notna(value) else 0

occurrences_enriched = (
    df_occurrences
    .merge(transect_meta, on="TransectUID", how="left")
    .dropna(subset=["Taxon Label", "habitat", "Pre: On old reserve?"])
)
occurrences_enriched = occurrences_enriched[
    occurrences_enriched["Pre: On old reserve?"] == "Yes"
]
occurrences_enriched = occurrences_enriched[
    occurrences_enriched["habitat"].isin(habitats)
]

occ_total_species = occurrences_enriched.groupby("Taxon Label").size()
occ_total_all = int(occ_total_species.sum()) if not occ_total_species.empty else 0
occ_total_percent = (
    occ_total_species.div(occ_total_all).mul(100)
    if occ_total_all
    else occ_total_species.astype(float)
).replace([np.inf, -np.inf], np.nan).fillna(0)

occ_by_hab_species = occurrences_enriched.groupby(["Taxon Label", "habitat"]).size()
occ_total_by_hab = occurrences_enriched.groupby("habitat").size()
occ_percent_by_hab = (
    occ_by_hab_species.div(occ_total_by_hab, level="habitat").mul(100)
    if not occ_total_by_hab.empty
    else occ_by_hab_species.astype(float)
).replace([np.inf, -np.inf], np.nan).fillna(0)

mni_enriched = (
    group_mni
    .merge(transect_meta, on="TransectUID", how="left")
    .dropna(subset=["Taxon", "habitat", "Pre: On old reserve?"])
)
mni_enriched = mni_enriched[mni_enriched["Pre: On old reserve?"] == "Yes"]
mni_enriched = mni_enriched[mni_enriched["habitat"].isin(habitats)]
mni_enriched = mni_enriched[~mni_enriched["Taxon"].isin(EXCLUDED_MNI_TAXA)]

mni_total_species = mni_enriched.groupby("Taxon")["Group MNI"].sum()
mni_total_all = int(mni_total_species.sum()) if not mni_total_species.empty else 0
mni_total_percent = (
    mni_total_species.div(mni_total_all).mul(100)
    if mni_total_all
    else mni_total_species.astype(float)
).replace([np.inf, -np.inf], np.nan).fillna(0)

mni_by_hab_species = mni_enriched.groupby(["Taxon", "habitat"])["Group MNI"].sum()
mni_total_by_hab = mni_enriched.groupby("habitat")["Group MNI"].sum()
mni_percent_by_hab = (
    mni_by_hab_species.div(mni_total_by_hab, level="habitat").mul(100)
    if not mni_total_by_hab.empty
    else mni_by_hab_species.astype(float)
).replace([np.inf, -np.inf], np.nan).fillna(0)

column_tuples = []
for habitat in [total_label] + habitats:
    for metric in ["Occurrence", "MNI"]:
        for stat in ["n", "%"]:
            column_tuples.append((habitat, metric, stat))
columns = pd.MultiIndex.from_tuples(
    column_tuples, names=["Habitat", "Measure", "Statistic"]
)

species_order = occ_total_species.sort_values(ascending=False).index.tolist()
for taxon in mni_total_species.index:
    if taxon not in species_order:
        species_order.append(taxon)

summary_rows = []
for taxon in species_order:
    row = {}
    occ_total_value = occ_total_species.get(taxon, 0)
    row[(total_label, "Occurrence", "n")] = _safe_int(occ_total_value)
    row[(total_label, "Occurrence", "%")] = occ_total_percent.get(taxon, 0.0)
    if taxon in EXCLUDED_MNI_TAXA:
        row[(total_label, "MNI", "n")] = pd.NA
        row[(total_label, "MNI", "%")] = pd.NA
    else:
        mni_total_value = mni_total_species.get(taxon, 0)
        row[(total_label, "MNI", "n")] = _safe_int(mni_total_value)
        row[(total_label, "MNI", "%")] = mni_total_percent.get(taxon, 0.0)
    for habitat in habitats:
        occ_hab_value = occ_by_hab_species.get((taxon, habitat), 0)
        row[(habitat, "Occurrence", "n")] = _safe_int(occ_hab_value)
        row[(habitat, "Occurrence", "%")] = occ_percent_by_hab.get((taxon, habitat), 0.0)
        if taxon in EXCLUDED_MNI_TAXA:
            row[(habitat, "MNI", "n")] = pd.NA
            row[(habitat, "MNI", "%")] = pd.NA
        else:
            mni_hab_value = mni_by_hab_species.get((taxon, habitat), 0)
            row[(habitat, "MNI", "n")] = _safe_int(mni_hab_value)
            row[(habitat, "MNI", "%")] = mni_percent_by_hab.get((taxon, habitat), 0.0)
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows, index=species_order, columns=columns)
summary_df.index.name = "Species"

percent_columns = [col for col in summary_df.columns if col[2] == "%"]
count_columns = [col for col in summary_df.columns if col[2] == "n"]

summary_df[percent_columns] = summary_df[percent_columns].applymap(
    lambda x: round(float(x), 2) if isinstance(x, numbers.Real) and not pd.isna(x) else x
)
summary_df[count_columns] = summary_df[count_columns].applymap(
    lambda x: int(x) if isinstance(x, numbers.Real) and not pd.isna(x) else x
)

total_row = {}
total_row[(total_label, "Occurrence", "n")] = occ_total_all
total_row[(total_label, "Occurrence", "%")] = 100.0 if occ_total_all else 0.0
total_row[(total_label, "MNI", "n")] = mni_total_all
total_row[(total_label, "MNI", "%")] = 100.0 if mni_total_all else 0.0
for habitat in habitats:
    occ_total_hab = _safe_int(occ_total_by_hab.get(habitat, 0))
    mni_total_hab = _safe_int(mni_total_by_hab.get(habitat, 0))
    total_row[(habitat, "Occurrence", "n")] = occ_total_hab
    total_row[(habitat, "Occurrence", "%")] = 100.0 if occ_total_hab else 0.0
    total_row[(habitat, "MNI", "n")] = mni_total_hab
    total_row[(habitat, "MNI", "%")] = 100.0 if mni_total_hab else 0.0

total_df = pd.DataFrame([total_row], index=["Total occurrences"], columns=columns)
summary_df = pd.concat([total_df, summary_df])

output_path = Path("../data/export/excel/mni_occurrence_summary_by_habitat.xlsx")
output_path.parent.mkdir(parents=True, exist_ok=True)
summary_df.to_excel(output_path, sheet_name="Summary", merge_cells=False)

summary_df

/tmp/ipykernel_417/2518691139.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  summary_df[percent_columns] = summary_df[percent_columns].applymap(
/tmp/ipykernel_417/2518691139.py:135: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  summary_df[count_columns] = summary_df[count_columns].applymap(


Habitat            eastern OPC                      grass closed          \
Measure             Occurrence           MNI          Occurrence           
Statistic                    n       %     n      %            n       %   
Total occurrences          580  100.00   373  100.0          243  100.00   
zebra                      156   26.90   124  33.24           83   34.16   
Mammalia indet             149   25.69     0    0.0           44   18.11   
warthog                     47    8.10    38  10.19           23    9.47   
Bovidae (medium)            31    5.34    28   7.51           18    7.41   
giraffe                     29    5.00    25    6.7           12    4.94   
Thompson's gazelle          23    3.97    20   5.36           19    7.82   
buffalo                     23    3.97    21   5.63            4    1.65   
Bovidae (small)             22    3.79    21   5.63            9    3.70   
Bovidae (large)             20    3.45    19   5.09            6    2.47   
impala                      18    3.10    18   4.83            3    1.23   
waterbuck                   17    2.93    19   5.09            3    1.23   
eland                       11    1.90    11   2.95            2    0.82   
Grant's gazelle              8    1.38     9   2.41            6    2.47   
hartebeest                   6    1.03     6   1.61            4    1.65   
cow (domestic)               5    0.86     6   1.61            4    1.65   
ungulate                     3    0.52     0    0.0            0    0.00   
hare                         2    0.34     2   0.54            1    0.41   
Aves (medium)                2    0.34  <NA>   <NA>            1    0.41   
Aves (small)                 2    0.34  <NA>   <NA>            0    0.00   
elephant                     2    0.34     2   0.54            0    0.00   
Rhinocerotidae               1    0.17     1   0.27            0    0.00   
black rhinoceros             1    0.17     1   0.27            1    0.41   
spotted hyaena               1    0.17     1   0.27            0    0.00   
reedbuck                     1    0.17     1   0.27            0    0.00   

Habitat                         shrubs open                       \
Measure              MNI         Occurrence           MNI          
Statistic              n      %           n       %     n      %   
Total occurrences    167  100.0         213  100.00   125  100.0   
zebra                 64  38.32          50   23.47    40   32.0   
Mammalia indet         0    0.0          70   32.86     0    0.0   
warthog               18  10.78          21    9.86    17   13.6   
Bovidae (medium)      15   8.98           9    4.23     9    7.2   
giraffe               10   5.99           8    3.76     7    5.6   
Thompson's gazelle    16   9.58           4    1.88     4    3.2   
buffalo                3    1.8          14    6.57    13   10.4   
Bovidae (small)        8   4.79           6    2.82     6    4.8   
Bovidae (large)        6   3.59           7    3.29     6    4.8   
impala                 3    1.8          10    4.69    10    8.0   
waterbuck              4    2.4           5    2.35     5    4.0   
eland                  2    1.2           2    0.94     2    1.6   
Grant's gazelle        7   4.19           1    0.47     1    0.8   
hartebeest             4    2.4           2    0.94     2    1.6   
cow (domestic)         5   2.99           1    0.47     1    0.8   
ungulate               0    0.0           0    0.00     0    0.0   
hare                   1    0.6           0    0.00     0    0.0   
Aves (medium)       <NA>   <NA>           0    0.00  <NA>   <NA>   
Aves (small)        <NA>   <NA>           1    0.47  <NA>   <NA>   
elephant               0    0.0           0    0.00     0    0.0   
Rhinocerotidae         0    0.0           1    0.47     1    0.8   
black rhinoceros       1    0.6           0    0.00     0    0.0   
spotted hyaena         0    0.0           1    0.47     1    0.8   
reedbuck               0    0.0    